# Projection of the risk free curve and recalibration inside OSEM

The purpose of this workbook is to showcase the calibration methodology inside OSEM. The key to the calibration is the Smith Wilson algorithm commonly used in insurance. This algorithm allows a continious approximation of arbitrary maturities based on a subset of available maturities. This example is split onto 3 parts. 

 - The first part shows how the necessary parameters and curves are imported from input files. 
 - Second part shows how the curves are generated and the Smith-Wilson calibrations derived. 
 - The last part shows how the saved calibration can be used to derive an the discount rates for any maturity.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick

In [2]:
from ImportData import import_SWEiopa
from CurvesClass import Curves

## Importing files

The parameters and the current risk free curve are provided as input:

 - Parameters.csv; Parameters related to the run 
 - EIOPA_param_file.csv; Assumed yield curve at modelling date and relevant maturities
 - EIOPA_curves_file.csv; Parameters related to the EIOPA time 0 calibration
 
 
 The exact location of the EIOPA files is encoded in the Parameters input file. 

In [3]:
paramfile = pd.read_csv("Input\Parameters.csv", index_col="Parameter")

The location of the two EIOPA files are:

In [4]:
selected_param_file = paramfile["Value"].loc["EIOPA_param_file"]
selected_curves_file = paramfile["Value"].loc["EIOPA_curves_file"]

The risk free curve belongs to the following country:

In [5]:
country = paramfile["Value"].loc["country"]

Import all necessary parameters:

In [6]:
[maturities_country, curve_country, extra_param, Qb] = import_SWEiopa(selected_param_file, selected_curves_file, country)

## Preparation and treatment of the curves

The OSEM does multiple steps in order to make it easier for OSEM to access discount curves at any point in time and for arbitrary maturities. All steps are performed and saved within the Curves class.

### The curves class

The curve class object contains all the data necessary to run the model.

In [7]:
# ultimate forward rate
ufr = extra_param["UFR"]/100

# Numeric precision of the optimisation
precision = float(paramfile["Value"].loc["Precision"])

# Targeted distance between the extrapolated curve and the ultimate forward rate at the convergence point
tau = float(paramfile["Value"].loc["Tau"])# 1 basis point

modelling_date = paramfile.loc["Modelling_Date"]

# Number of projection years
n_years = int(paramfile.loc["n_proj_years"][0])

In [8]:
curves = Curves(ufr, precision, tau, modelling_date, country)

After the Curves class is initiated, there are a series if calibrations that needs to be performed.

 - Saving of the input term structure
 - Calculation of the forward rates
 - Projection of the forward spot rates
 - Calculation of the Smith-Wilson calibrations for each period

### Save the input term structure

SetObservedTermStructure sets the liquid maturities and the coresponding yields to the m_obs and r_obs property of the Curves class. 

In [9]:
curves.SetObservedTermStructure(maturity_vec=curve_country.index.tolist(), yield_vec=curve_country.values)

### Calculate 1 year forward rates

Using the time 0 spot curve provided as input, the yearly forward rates can be calculated from the annually reported yield curve
$$
fw_{EIOPA,1} = y_1
$$

otherwise:

$$
fw_{EIOPA,i} = \frac{(1 + y_i)^{-i}}{(1 + y_{i-1})^{-(i-1)}} 
$$

The forward rates will be used to calculate forward spot curves.

Where:

 - $i$ is the year (Ex. 1,2,3, $\dots$)
 - $y_i$ is the yield for the year $i$
 - $fw_{EIOPA,i}$ is the 1-year forward rate between years $i-1$ and $i$

In [10]:
curves.CalcFwdRates()

### Forward yield curve

The forward yield curve can be calculated by using the 1-year forward rates from the suitable moment onwards. The spot yield curve is calculated for each projection period. For the projection period $i$, the yield curve can be calculated as:

$$
y^j_{i-j} = \prod_{k=j}^i\big(1+fw_{EIOPA}(k)\big)^{-(k-j)}
$$

Where:
 - $j$ is the modelling period at which the model is performing the calculations 
 - $i-j$ is the time period within the modelling period
 - $y^j_{k}$ is the yield $k$ years after the modelling period $j$

In [11]:
curves.ProjectForwardRate(n_years)

## Calibrate Smith-Wilson on each yield curve

Each forward spot curve is used to calibrate the SW algorithm. This calibration can then be used to generate yield corves of arbitrary maturities. The calibration is split on two parts. First one calibrates the curve at time 0

### Calibration of the interpolation/extrapolation algorithm

In [12]:
curves.CalibrateProjected(n_years, 0.05, 0.5, 1000)

## Example 1

Is the yield curve equal to the curve used for the calibration?

In [13]:
#r_obs_est = curves.SWExtrapolate(m_obs, m_obs, curves.b[NameOfYear_3][:-(ProjYear)], curves.ufr, curves.alpha[NameOfYear_4][0])

In [14]:
#fig, ax1 = plt.subplots(1, 1)
#ax1.scatter(m_obs, r_obs, label="Calculated", marker="x")
#ax1.plot(m_obs, r_obs_est)
#ax1.set_ylabel("yield")
#ax1.set_title('Recalculated & provided yield curve')
#ax1.set_xlabel("time")
#ax1.legend()
#ax1.yaxis.set_major_formatter(mtick.PercentFormatter())
#fig.set_figwidth(6)
#fig.set_figheight(3)
#plt.show()

## Example 2

The future yield curve for arbitrary maturities.

Assuming the algorithm is processing the year 3. A hypothetical asset has a cash flow at year fractions:

$$ \{0.7, 1.2, 2.1, 3.543\} $$

To make it possible to calculate the present value, the coresponding dicount rates are calculated:


In [16]:
desired_mat = np.array([0.7,1.2,1.3543])

### Yield calculation


The yield in each modellig period can be interpolated/extrapolated using the Smith-Wilson calibration vector calculated above. This allows OSEM to apply continious yields.

In [17]:
curves.RetrieveRates(3, desired_mat, "Yield")

array([0.02618662, 0.026155  , 0.02614528])

### Capitalisation factor calculation


The capitalisation factor can be calculated as:

$$
c_i(t) = \big(1+ y_i(t)\big)^{t} 
$$


In [18]:
curves.RetrieveRates(3, desired_mat, "Capitalisation")

array([1.01825944, 1.03146752, 1.03557165])

### Discount factor calculation

The discount factors can be calculated from the yield:

$$
d_i(t) = \big(1+ y_i(t)\big)^{-t} 
$$

Where:
 - $t$ is time increment from the modelling period (Ex. 1.5 is 1 year and 6 months after the modelling period
 - $y_i(t)$ is the annualized yield for maturity t after the modelling period $i$
 - $c_i(t)$ is the capitalisation factor for 1 unit, until $t$ years after the modelling period $i$

 - $d_i(t)$ is the discount factor for a cash flow of value 1, occuring $t$ years after the modelling period $i$, discounted to the modelling period $i$


In [19]:
curves.RetrieveRates(3, desired_mat, "Discount")

array([0.98206799, 0.96949248, 0.96565023])